In [ ]:
pip install azure-ai-documentintelligence azure-storage-blob python-dotenv pandas

In [ ]:
# Credenciais da Inteligência de Documentos do Azure
AZURE_DOC_INT_ENDPOINT="SEU_ENDPOINT_AQUI"
AZURE_DOC_INT_KEY="SUA_CHAVE_AQUI"

# String de Conexão do Azure Blob Storage
AZURE_STORAGE_CONNECTION="SUA_STRING_DE_CONEXAO_AQUI"
CONTAINER_NAME="NOME_DO_SEU_CONTAINER_AQUI"

In [ ]:
import os
import sqlite3
import pandas as pd
from datetime import datetime
from typing import Dict, Any, Optional, Tuple

# Bibliotecas do Azure
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest

# Carregador de variáveis de ambiente
from dotenv import load_dotenv

# Carrega as variáveis do arquivo .env
load_dotenv()

class AppConfig:
    """Classe para gerenciar as configurações e credenciais da aplicação."""
    ENDPOINT = os.getenv("AZURE_DOC_INT_ENDPOINT")
    KEY = os.getenv("AZURE_DOC_INT_KEY")
    STORAGE_CONNECTION_STRING = os.getenv("AZURE_STORAGE_CONNECTION")
    STORAGE_CONTAINER_NAME = os.getenv("CONTAINER_NAME")
    DATABASE_FILE = "credit_card_records.db"

In [ ]:
class DatabaseManager:
    """Gerencia as operações do banco de dados SQLite para os registros de cartões."""
    def __init__(self, db_file: str):
        self.db_path = db_file
        self._initialize_database()

    def _get_connection(self):
        return sqlite3.connect(self.db_path)

    def _initialize_database(self):
        """Cria a tabela de cartões de crédito se ela não existir."""
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS credit_cards (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    cardholder_name TEXT,
                    card_number TEXT UNIQUE,
                    expiration_date TEXT,
                    issuing_bank TEXT,
                    is_valid BOOLEAN,
                    processed_timestamp TEXT
                )
            """)
            conn.commit()

    def add_card_record(self, card_data: Dict[str, Any]) -> int:
        """Adiciona um novo registro de cartão ao banco de dados."""
        query = """
        INSERT INTO credit_cards (
            cardholder_name, card_number, expiration_date, issuing_bank, is_valid, processed_timestamp
        ) VALUES (?, ?, ?, ?, ?, ?)
        """
        with self._get_connection() as conn:
            cursor = conn.cursor()
            cursor.execute(query, (
                card_data.get('card_name'),
                card_data.get('card_number'),
                card_data.get('expiry_date'),
                card_data.get('bank_name'),
                card_data.get('is_valid'),
                card_data.get('processed_at')
            ))
            conn.commit()
            return cursor.lastrowid

    def find_card_by_number(self, card_number: str) -> Optional[Dict[str, Any]]:
        """Busca um cartão pelo seu número para evitar duplicatas."""
        query = "SELECT * FROM credit_cards WHERE card_number = ?"
        with self._get_connection() as conn:
            conn.row_factory = sqlite3.Row
            cursor = conn.cursor()
            cursor.execute(query, (card_number,))
            record = cursor.fetchone()
            return dict(record) if record else None

    def get_all_records(self) -> pd.DataFrame:
        """Retorna todos os registros do banco de dados como um DataFrame do Pandas."""
        query = "SELECT * FROM credit_cards"
        with self._get_connection() as conn:
            df = pd.read_sql_query(query, conn)
        return df

In [ ]:
class BlobStorageUploader:
    """Responsável pelo upload de imagens para o Azure Blob Storage."""
    def __init__(self, conn_string: str, container_name: str):
        self.blob_service_client = BlobServiceClient.from_connection_string(conn_string)
        self.container_name = container_name

    def upload_image_from_path(self, local_file_path: str, blob_name: Optional[str] = None) -> str:
        """
        Faz o upload de uma imagem e retorna sua URL.
        Se blob_name não for fornecido, usa o nome do arquivo local.
        """
        if not blob_name:
            blob_name = os.path.basename(local_file_path)

        blob_client = self.blob_service_client.get_blob_client(
            container=self.container_name, blob=blob_name
        )

        with open(local_file_path, "rb") as data:
            blob_client.upload_blob(data, overwrite=True)

        return blob_client.url

In [ ]:
class DocumentAnalyzer:
    """Extrai informações de cartões de crédito usando a Inteligência de Documentos do Azure."""
    def __init__(self, endpoint: str, key: str):
        self.client = DocumentIntelligenceClient(endpoint, AzureKeyCredential(key))

    def extract_card_details(self, image_url: str) -> Optional[Dict[str, Any]]:
        """Analisa a imagem de um cartão e retorna os campos extraídos."""
        poller = self.client.begin_analyze_document(
            "prebuilt-creditCard", AnalyzeDocumentRequest(url_source=image_url)
        )
        result = poller.result()

        if not result.documents:
            return None

        # Processa o primeiro documento encontrado
        document = result.documents[0]
        extracted_fields = {}

        for key, field in document.fields.items():
            if field.type == 'array':
                extracted_fields[key] = [item.content for item in field.value_array] if field.value_array else None
            else:
                extracted_fields[key] = field.content

        return extracted_fields

In [ ]:
def process_and_validate_card(image_path: str):
    """
    Executa o fluxo completo: upload, análise, validação e armazenamento.
    """
    # 1. Inicializa os serviços
    config = AppConfig()
    uploader = BlobStorageUploader(config.STORAGE_CONNECTION_STRING, config.STORAGE_CONTAINER_NAME)
    analyzer = DocumentAnalyzer(config.ENDPOINT, config.KEY)
    db_manager = DatabaseManager(config.DATABASE_FILE)

    # 2. Faz o upload da imagem para o Blob Storage
    print(f"Fazendo upload de '{image_path}'...")
    image_url = uploader.upload_image_from_path(image_path)
    print(f"Upload concluído. URL: {image_url}")

    # 3. Extrai informações da imagem
    print("Analisando a imagem do cartão...")
    extracted_data = analyzer.extract_card_details(image_url)
    if not extracted_data:
        print("Nenhuma informação pôde ser extraída do documento.")
        return

    # 4. Valida os campos essenciais
    card_number = extracted_data.get('CardNumber')
    card_holder = extracted_data.get('CardHolderName')
    expiration_date = extracted_data.get('ExpirationDate')
    issuing_bank = extracted_data.get('IssuingBank')

    is_valid_card = all([card_number, card_holder, expiration_date, issuing_bank])

    if is_valid_card:
        print("\n--- Cartão Válido ---")
    else:
        print("\n--- Cartão Inválido: Faltam informações essenciais. ---")

    print(f"  - Nome do Titular: {card_holder or 'Não detectado'}")
    print(f"  - Número do Cartão: {card_number or 'Não detectado'}")
    print(f"  - Validade: {expiration_date or 'Não detectado'}")
    print(f"  - Banco: {issuing_bank or 'Não detectado'}")

    # 5. Verifica duplicatas e salva no banco de dados, se aplicável
    if is_valid_card:
        if db_manager.find_card_by_number(card_number):
            print(f"\nAVISO: O cartão de número {card_number} já existe no banco de dados.")
        else:
            record_to_save = {
                'card_name': card_holder,
                'card_number': card_number,
                'expiry_date': expiration_date,
                'bank_name': issuing_bank,
                'is_valid': True,
                'processed_at': datetime.now().isoformat()
            }
            db_manager.add_card_record(record_to_save)
            print("\nSUCESSO: Novas informações do cartão foram salvas no banco de dados.")
    else:
        print("\nAVISO: As informações do cartão inválido não foram salvas.")

In [ ]:
valid_card_path = "caminho/para/cartao_valido.png" 

In [ ]:
invalid_card_path = "caminho/para/cartao_invalido.jpg" 

In [ ]:
# Inicializa o gerenciador e exibe todos os registros
db = DatabaseManager(AppConfig.DATABASE_FILE)
all_cards_df = db.get_all_records()

print("\n--- Registros Atuais no Banco de Dados ---")
if all_cards_df.empty:
    print("Nenhum registro encontrado.")
else:
    print(all_cards_df.to_string())